In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("../데이터자료/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_베이시스','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [2]:
df["1Y_베이시스대비"] =df["1Y_베이시스"]-df["1Y_베이시스"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

,종가_ex,1Y_베이시스,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_베이시스대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,
2012-08-01,1126.5,-76.0,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,-74.0,2.79,2.77,1134.00,2.5,2.0,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,-72.0,2.77,2.75,1138.00,2.4,2.0,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,-74.0,2.78,2.76,1128.50,2.4,-2.0,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,-73.0,2.77,2.75,1129.00,2.5,1.0,-0.01,-0.01,0.50,0.1


In [3]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns


Index(['종가_ex', '1Y_베이시스', '국고1년', '통안364일', 'Mid_ndf', 'M1_스왑포인트',
       '1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비',
       '전날 종가_ex'],
      dtype='object')

In [5]:
# 종속변수 독립변수 설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_베이시스', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]
df_Data.feature= df_Data.columns
df_Data.feature

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_47008\4034184235.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_베이시스', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트', '전날 종가_ex'], dtype='object')

In [6]:
# 스케일링 코드
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()

scaler.fit(df_Data)
df_Data_scaled= scaler.transform(df_Data)
df_scaled = pd.DataFrame (data = df_Data_scaled, columns=df_Data.feature)
df_scaled.index = df_Target.index

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.917e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        20:53:51   Log-Likelihood:                -7749.4
No. Observations:                2459   AIC:                         1.551e+04
Df Residuals:                    2452   BIC:                         1.555e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1134.8939      0.114   9938.025      0.000    1134.670    1135.118
1Y_베이시스       -0.2922      0.114     -2.552      0.011      -0.517      -0.068
국고1년대비         0.2004      0.116      1.731      0.084      -0.027       0.428
통안364일대비       0.0008      0.115      0.007      0.995      -0.226       0.227
Mid_ndf대비      0.6216      0.114      5.442      0.000       0.398       0.846
M1_스왑포인트      -0.1391      0.123     -1.127      0.260      -0.381       0.103
전날 종가_ex      55.2649      0.124    445.721      0.000      55.022      55.508
==============================================================================
Omnibus:                      146.528   Durbin-Watson:                   2.164
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              621.790
Skew:                          -0.001   Prob(JB):                    9.55e-136
Kurtosis:                       5.463   Cond. No.                         1.51
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
# p값높은 컬럼들 제외
df_Target = df["종가_ex"]
df_scaled = df_scaled[['1Y_베이시스', '국고1년대비', 'Mid_ndf대비','전날 종가_ex']]

In [10]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')
# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 5.877e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        20:54:41   Log-Likelihood:                -7750.0
No. Observations:                2459   AIC:                         1.551e+04
Df Residuals:                    2454   BIC:                         1.554e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1134.8939      0.114   9939.506      0.000    1134.670    1135.118
1Y_베이시스       -0.2928      0.114     -2.558      0.011      -0.517      -0.068
국고1년대비         0.1978      0.114      1.728      0.084      -0.027       0.422
Mid_ndf대비      0.6218      0.114      5.444      0.000       0.398       0.846
전날 종가_ex      55.3178      0.115    482.138      0.000      55.093      55.543
==============================================================================
Omnibus:                      144.743   Durbin-Watson:                   2.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              607.215
Skew:                           0.004   Prob(JB):                    1.40e-132
Kurtosis:                       5.434   Cond. No.                         1.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# p값높은 컬럼들 제외
df_Target = df["종가_ex"]
df_scaled = df_scaled[['1Y_베이시스',  'Mid_ndf대비','전날 종가_ex']]

In [14]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_scaled, has_constant='add')
# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 7.830e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        21:10:49   Log-Likelihood:                -7751.5
No. Observations:                2459   AIC:                         1.551e+04
Df Residuals:                    2455   BIC:                         1.553e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1134.8939      0.114   9935.489      0.000    1134.670    1135.118
1Y_베이시스       -0.2959      0.115     -2.584      0.010      -0.520      -0.071
Mid_ndf대비      0.6248      0.114      5.469      0.000       0.401       0.849
전날 종가_ex      55.3317      0.114    483.249      0.000      55.107      55.556
==============================================================================
Omnibus:                      150.498   Durbin-Watson:                   2.165
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              654.909
Skew:                          -0.004   Prob(JB):                    6.14e-143
Kurtosis:                       5.528   Cond. No.                         1.07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
X_train, X_test, y_train, y_test = train_test_split(feature_add, df_Target,test_size=0.2, random_state=15 )

lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse

5.838727602421525

In [17]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, feature_add, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.306610103025923,
 array([ 4.40826373,  6.69742363,  5.67100534, 19.55392294,  5.20243488]),
 array([ -19.43278907,  -44.85548332,  -32.16030151, -382.35590226,
         -27.06532872]))